In [ ]:
#### IMPORTS ####
using ProgressMeter
using Plots
using LaTeXStrings
using NPZ
include("dWaveRgFlow.jl")

In [ ]:
num_kspace_half = 30
num_kspace = 2 * num_kspace_half + 1
kx_arr = range(-pi, stop=pi, length=num_kspace)
@time kondoJArray, dispersionArray = main(num_kspace_half, 1, 0.5, 0.1)
dosArray = getDOS(num_kspace, dispersionArray)
antinode = num_kspace_half + 1
node = div(num_kspace_half, 2) * (num_kspace + 1)

plots1 = []
plots2 = []
for i in [1, div(num_kspace_half, 2)+div(num_kspace_half, 3), num_kspace_half+1]
    J_antinode = reshape(kondoJArray[antinode,:,i], (num_kspace, num_kspace))
    J_node = reshape(kondoJArray[node,:,i], (num_kspace, num_kspace))
    push!(plots1, heatmap(kx_arr, kx_arr, reshape(J_antinode, (num_kspace, num_kspace))))
    push!(plots2, heatmap(kx_arr, kx_arr, reshape(J_node, (num_kspace, num_kspace))))
end
p2 = plot(plots1..., layout=(3, 1), size=(600, 400 * 3), thickness_scaling=1.2, xlims = (-1.1*pi, 1.1*pi), ylims = (-1.1*pi, 1.1*pi), xlabel="\$k_x\$", ylabel="\$k_y\$")
savefig(p2, "figAntiNode.pdf")
p3 = plot(plots2..., layout=(3, 1), size=(600, 400 * 3), thickness_scaling=1.2, xlims = (-1.1*pi, 1.1*pi), ylims = (-1.1*pi, 1.1*pi), xlabel="\$k_x\$", ylabel="\$k_y\$")
savefig(p3, "figNode.pdf")